In [ ]:
# J'importe mes bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Bibiliothèques de ML
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import NearestNeighbors

# Import pipeline
from sklearn.pipeline import Pipeline

# Import outil standardisation de la donnée
from sklearn.preprocessing import StandardScaler

# Le module pour spliter le modèle
from sklearn.model_selection import train_test_split

# Import des métriques
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error


# Gestion des warnings
import warnings

: 

In [ ]:
url = "https://raw.githubusercontent.com/halekss/.fivesensefilms.io/refs/heads/main/Films.csv"
films = pd.read_csv(url, sep=";")

In [ ]:
def quick_explore(dataframe):
    """
    Fonction d'exploration rapide des données.
    """
    # Afficher quelques lignes
    print("###### Observer des lignes ######")
    display(dataframe.head(10), "\n")


    # Afficher le nombre de lignes et colonnes
    print("###### Dimensions du dataset ######")
    print(f"Lignes : {dataframe.shape[0]}, Colonnes : {dataframe.shape[1]}\n")

    # Informations sur les colonnes
    print("###### Informations sur les colonnes ######")
    print(dataframe.info(), "\n")

    # Les nom de colonnes
    print(list(dataframe.columns))


    # Nombre de valeurs uniques par colonne
    print("###### Nombre de valeurs uniques par colonne ######")
    print(dataframe.nunique(), "\n")

    # Description des colonnes numériques avec les déciles
    print("###### Description des colonnes numériques ######")
    stats = dataframe.describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).T
    print( stats.to_markdown(), "\n")

    # Nombre de valeurs manquantes par colonne
    print("###### Nombre de NaN par colonne ######")
    nan_nb = dataframe.isna().sum()
    print(nan_nb)

    # Pourcentage de valeurs manquantes par colonne
    print("###### Pourcentage de NaN par colonne ######")
    nan_percentage = (dataframe.isnull().sum() / len(dataframe)) * 100
    only_nan  = nan_percentage[nan_percentage > 0].sort_values(ascending=False)
    print(only_nan, "\n")

    # Nombre de doublons
    print("###### Nombre de doublons ######")
    print(dataframe.duplicated().sum(), "doublon(s) trouvé(s).\n")

print(quick_explore(films))

In [ ]:
display(films.head(10))

In [ ]:
films["URL complète"]=films["URL complète"].fillna("Non renseigné")
films["Arrière plan film"]=films["Arrière plan film"].fillna("Non renseigné")

In [ ]:
films['Résolution'] = films['Résolution'].str.replace(',', '.', regex=False).astype(float)
#films['Résolution'].astype(int)
films['Résolution'].describe()

In [ ]:
films['Résolution'].fillna(films['Résolution'].median(), inplace=True)
display(films['Résolution'].isna().sum())

In [ ]:
films['Nombre de votants'].fillna(0, inplace=True)

In [ ]:
films.isna().sum()

In [ ]:
films.columns

In [ ]:
films.drop(columns=["Genres","Id des genres", "Id Youtube", "Arrière plan film", "Arrière plan film", "Officiel", "Résolution", "Slogan", "Type de vidéo", "Date de mise en ligne"], inplace=True)

In [ ]:
# Je décide de créer une fonction pour concaténer le type 1 et le type 2 (pour rappel 48% de valeurs manquantes sur la colonne 'Type 2')

def concatenate_type(row):
  genre1 = row['Genre Principal']
  genre2 = row['Genre secondaire']
  genre3 = row['Genre tertiaire']

  # Si mon type 2 n'est pas un nan alors retournes moi type1,type2:
  if genre2 != 'Non renseigné':
    genre1 = genre1 + ',' + genre2
    if genre3 != 'Non renseigné':
        genre1 = genre1 + ',' + genre3
    return genre1
  # sinon tu me retoures uniquement le type 1 (dans le cas ou type 2 est un nan)
  else:
    return genre1

In [ ]:
films['Genres'] = films.apply(concatenate_type, axis=1)

In [ ]:
display(films.sample(15))

In [ ]:
films.drop(columns=['Genre Principal', 'Genre secondaire', 'Genre tertiaire'],inplace=True)

In [ ]:
import ast

def extraire_mots_cles(x):
    try:
        d = ast.literal_eval(x)     # transforme "{212: 'london', ...}" en vrai dict Python
        if isinstance(d, dict):
            return ", ".join(d.values())
    except:
        return "Non renseigné"

films["Mots clés propre"] = films["Mots clés"].apply(extraire_mots_cles)

In [ ]:
films.drop(columns=['Mots clés'], inplace=True)

In [ ]:
films['Pays d\'origine'] = (
    films['Pays d\'origine']
    .str.replace(r"[\[\]']", "", regex=True)  # enlève crochets et apostrophes
    .str.strip()
)


In [ ]:
import ast

def extraire_pays(x):
    try:
        liste = ast.literal_eval(x)  # convertit la chaîne en vraie liste de dict
        if isinstance(liste, list):
            # on prend uniquement la valeur "name" dans chaque dictionnaire
            noms = [d.get("name", "") for d in liste if isinstance(d, dict)]
            return ", ".join(noms)
    except:
        return 'Non renseigné'

films["Pays_production_propre"] = films["Pays d'origine de la production"].apply(extraire_pays)


In [ ]:
def extraire_noms_societes(x):
    try:
        data = ast.literal_eval(x)  # transforme la chaîne en vraie liste
        if isinstance(data, list):
            noms = [d.get("name", "") for d in data if isinstance(d, dict)]
            return ", ".join(noms)
    except:
        return "Non renseigné"

films["Société_production_name"] = films["Société de production"].apply(extraire_noms_societes)


In [ ]:
def extraire_logo_path(x):
    try:
        data = ast.literal_eval(x)  # convertit texte → liste de dict
        if isinstance(data, list):
            logos = [d.get("logo_path") for d in data if isinstance(d, dict)]
            # enlever les None et ne conserver que les vraies valeurs
            logos = [l for l in logos if l]
            return ", ".join(logos) if logos else None
    except:
        return "Non renseigné"

films["Société_production_logo"] = films["Société de production"].apply(extraire_logo_path)


In [ ]:
def construire_urls(cell):
    if not isinstance(cell, str):
        return 'Non renseigné'
    base = "https://image.tmdb.org/t/p/w300"
    logos = [x.strip() for x in cell.split(",")]
    return ", ".join([base + x for x in logos if x])

films["logo_urls"] = films["Société_production_logo"].apply(construire_urls)


In [ ]:
films.columns

In [ ]:
films.drop(columns=['Société_production_logo','Société de production', "Pays d'origine de la production" ],inplace=True)

In [ ]:
films["Popularité"] = (
    films["Popularité"]
    .astype(str)
    .str.replace(",", ".", regex=False)
)

films["Popularité"] = pd.to_numeric(films["Popularité"], errors="coerce")


In [ ]:
films["Moyenne des votes"] = (
    films["Moyenne des votes"]
    .astype(str)
    .str.replace(",", ".", regex=False)
)

films["Moyenne des votes"] = pd.to_numeric(films["Moyenne des votes"], errors="coerce")


In [ ]:
def fill_with_group_median(s):
    s_non_na = s.dropna()
    if s_non_na.empty:   # Aucun budget/recette valide dans ce pays
        return s         # On laisse tel quel
    med = s_non_na.median()
    return s.fillna(med)

films["Budget"] = films.groupby("Pays_production_propre")["Budget"].transform(fill_with_group_median)
films["Recettes"] = films.groupby("Pays_production_propre")["Recettes"].transform(fill_with_group_median)


In [ ]:
films["Budget"] = pd.to_numeric(films["Budget"], errors="coerce")
films["Recettes"] = pd.to_numeric(films["Recettes"], errors="coerce")

In [ ]:
films.isna().sum()

In [ ]:
def fill_with_group_median(s):
    s_non_na = s.dropna()
    if s_non_na.empty:
        return s
    med = s_non_na.median()
    return s.fillna(med)

films["Budget"] = films.groupby("Pays_production_propre")["Budget"].transform(fill_with_group_median)
films["Recettes"] = films.groupby("Pays_production_propre")["Recettes"].transform(fill_with_group_median)


In [ ]:
# Construction de la matrice de similarité

films["Budget"] = films["Budget"].fillna(films["Budget"].median())
films["Recettes"] = films["Recettes"].fillna(films["Recettes"].median())

In [ ]:
# Observation des index sur les colonnes restantes pour la construction de la matrice de similarité
# films.columns

films.columns

In [ ]:
# Ré-indexation des colonnes pour une meilleure lisibilité.

films.reindex(columns=['tconst', 'Id_TMDB', 'Titre', 'Titre Original', 'Date de sortie',
       'Durée', 'Budget', 'Recettes', 'Genres', "Pays d'origine", "Langue Originale",'Mots clés propre',
       'Moyenne des votes', 'Nombre de votants', 'Popularité', 'Résumé',
       'URL complète', 'Affiche du Film',
       'Pays_production_propre', 'Société_production_name', 'logo_urls'])

In [ ]:
# Renommage des colonnes pour un affichage plus lisible

films.rename(columns={'URL complète': 'Lien_vidéo', 
       'Pays_production_propre':'Pays_production', 
       'Société_production_name':'Société_production', 
        'Mots clés propre':'Mots clés',
       'logo_urls':'Logo'}, inplace=True)

In [ ]:
# Dernière lecture des données.

films.info()

In [ ]:
display(films)

La donnée est nettoyée, les valeurs vides traités, les colonnes non pertinente supprimé.

Le dataframe est à présent près pour le traitement des dummies

In [ ]:
# sur une copie du datframe je stock les comptage unitaire de chaque mots clés
films_copie = films['Mots clés'].str.split(',').explode().value_counts().sort_values(ascending=False)
display(films_copie.describe(percentiles= (0.8,0.9 , 0.95, 0.96, 0.97)))

In [ ]:
# 1. On éclate tous les mots clés film par film
mot_cle_individuel = (
    films['Mots clés']
    .dropna()
    .str.split(',')      # "a,b,c" -> ["a","b","c"]
    .explode()           # 1 mot clé par ligne
    .str.strip()         # on enlève les espaces
)

# 2. On enlève la pseudo valeur "Non renseigné"
mot_cle_individuel = mot_cle_individuel[mot_cle_individuel != 'Non renseigné']

# 3. Fréquence de chaque mot clé
mot_cle_comptage = mot_cle_individuel.value_counts()

# 4. Seuil du 95e percentile
top5 = mot_cle_comptage.quantile(0.95)

# 5. Ensemble des mots clés à conserver (top 5 %)
top_mot_cle = mot_cle_comptage[mot_cle_comptage >= top5].index
top_mot_cle_set = set(top_mot_cle)  # plus rapide pour le test d'appartenance

display(top_mot_cle_set)

In [ ]:
def filter_top_keywords(cell):
    # cas NaN
    if pd.isna(cell):
        return np.nan
    
    # on découpe les mots clés de ce film
    lst = [k.strip() for k in cell.split(',')]
    
    # on ne garde que ceux du top 5 %
    kept = [k for k in lst if k in top_mot_cle_set]
    
    # si un film n'a aucun mot clé "top 5 %"
    if not kept:
        return np.nan    # ou '' si tu préfères
    return ','.join(kept)

films['Mots_cles_top5'] = films['Mots clés'].apply(filter_top_keywords)
display(films)

In [ ]:
# Encodage de la colonne 'Genres' en dummies pour usage binaire du ML
# films['Genres'].str.get_dummies(',')  # syntaxe lorsque l'on a plusieurs catégories dans la même cellule

dummies = films['Genres'].str.get_dummies(',')  # syntaxe lorsque l'on a plusieurs catégories dans la même cellule

display(dummies)

In [ ]:
# Je concate le df initial et le dummies

df_films = pd.concat([films,dummies], axis=1)

In [ ]:
# Même démarche que pour la colonne Genres avec pays_production

# films['pays_production'].str.get_dummies(',')  # syntaxe lorsque l'on a plusieurs catégories dans la même cellule

dummies_2 = films['Pays_production'].str.get_dummies(',')  # syntaxe lorsque l'on a plusieurs catégories dans la même cellule

display(dummies_2)

In [ ]:
# Je concate le df initial et le dummies

df_films = pd.concat([films,dummies_2], axis=1).drop('Pays_production')

In [ ]:
# Même démarche que pour la colonne Genres avec mots_clés

# films['Mots clés'].str.get_dummies(',')  # syntaxe lorsque l'on a plusieurs catégories dans la même cellule

dummies_3 = films['Mots clés'].str.get_dummies(',')  # syntaxe lorsque l'on a plusieurs catégories dans la même cellule

display(dummies_3)

In [ ]:
# Je concate le df initial et le dummies

df_films = pd.concat([films,dummies_3], axis=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# Normalisation des données
scaler = MinMaxScaler()

# Correction de la colonne 'Moyenne des votes' au lieu de 'Moyenne des notes'
cols_scaler = ['Moyenne des votes', 'Popularité']
df_films[cols_scaler] = scaler.fit_transform(df_films[cols_scaler])

# Création du système d'entrainement et de test du machine learning
# Modèle retenu : K-Nearest Neighbors 80% Entrainement et 20% Test

# Supposons que X et y sont définis correctement
# X = df_films.drop(columns=['Moyenne des votes'])  # Exemple de définition
# y = df_films['Moyenne des votes']  # Exemple de définition

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, 
    train_size=0.8, 
    test_size=0.2, 
    random_state=42, 
    stratify=y)

# K en tant que nombre de voisins
K_voisins = 5

# Classement des voisins
knn = NearestNeighbors(n_neighbors=K_voisins)

# Entrainement du modèle et prédiction
knn.fit(X_train)
y_pred = knn.kneighbors(X_test)  # Utilisation de kneighbors pour obtenir les voisins

# Affichage des prédictions
print(y_pred)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=eb8103ec-0d85-46cb-8ff1-660cc3c3a559' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>